In [1]:
import pandas as pd
import re

def str_to_list(d):
  text = re.sub(r'[\[\'\]]', '', d)
  return text.split(", ")

In [2]:
import torchtext
from torchtext.legacy.data import Field
# text 필드
TEXT = Field(tokenize = str_to_list) # 내가 만든 토크나이즈

# label 필드
LABEL = Field(sequential = False) # label은 문장 단어 없음

In [3]:
from torchtext.legacy.data import TabularDataset

# 필드 지정 후 데이터셋 만들어야한다
# 데이터를 불러오면서 필드에서 정의했던 방식으로 토큰화 수행하고 기본적인 전처리 진행해준다
train, validation = TabularDataset.splits(
    path = './',
    train = 'train2.csv',
    validation = 'validation2.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)],
    skip_header = True
)
# format = 데이터 형식
# fields = Field 에서 지정한 것 불러와서 적용
# skip_header = 데이터 첫 번째 줄 무시

print(train[0].text, train[0].label)
print(validation[0].text, validation[0].label)

['김', '다나', '디자인', '기자', '남', '이야기', '를', '듣', '지', '않', '고', '라', '떼', '만', '계속', '이야기', '하', '는', '기업', '은', '꼰대', '로', '낙인찍히', '게', '되', 'ㅂ니다', '최', '태원', 'SK', '그룹', '회장', '의', '신', '기업가', '정신', '이', 'MZ', '세대', '직장인', '들', '의', '마음', '을', '홀리', '었', '다', 'SK', '그룹', '을', '재계', '서열', '2', '위', '로', '올려놓', '은', '양적', '성장', '이외', '에', '도', '수평적', '소통', '과', '사내', '복지', '에', '힘쓰', '는', '새', '경영', '철학', '을', '내세우', '면서', '다', '업계', '는', '창업', '2', '세대', '로', '꼽히', '는', '60', '대', '재벌', '총수', '의', '수평적', '화법', '이', '긍정적', '이', 'ㄴ', '평가', '를', '받', '고', '있', '다고', '분석', '하', 'ㄴ다', '10', '일', '업계', '에', '따르', '면', '최', '회장', '은', '최근', '꼰대', '에서', '벗', '어', '나', '기업', '의', '새', '역할', '을', '강조', '하', '는', '신', '기업가', '정신', '을', '경영', '화두', '로', '내세우', '었', '다', '이윤', '창출', '을', '넘', '어', '사회적', '공헌', '에', '기여', '하', '고', '기후', '변화', '환경', '오염', '등', '여러', '방면', '의', '문제', '를', '기업', '이', '적극', '해결', '하', '어야', '하', 'ㄴ다는', '주장', '이', '다', '최', '회장', '은', '한국', '사회', '에', '반', '기업', '정서', '가

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.vocab import Vectors
from torchtext.legacy.data import BucketIterator

# torchtext.vocab.Vectors(name = 벡터를 포함하는 파일 이름,
#                         cache = 캐시된 벡터의 디렉토리,
#                         url = 캐시에서 벡터를 찾을 수 없는 경우 다운로드할 url,
#                         )
# 

# 임베딩 벡터 불러오기
vectors = Vectors(name = './fasttext_newstoken')

# 단어장 만들기
TEXT.build_vocab(train,
                 vectors = vectors, min_freq = 1, max_size = None)


LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# train, validation set 을 지정한 배치 사이즈만큼 로드하여 각각 batch data를 생성
train_iter, validation_iter = BucketIterator.splits(
    datasets = (train, validation),
    batch_size = 8, # 각 batch 별 크기
    device = device,
    sort = False
)

print('임베딩 벡터 개수와 차원 : {}'.format(TEXT.vocab.vectors.shape))

임베딩 벡터 개수와 차원 : torch.Size([74334, 100])


In [5]:
# textCNN 정의
class TextCNN(nn.Module):
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        # '''
        # vocab_built : train 데이터로 생성한 단어장 인자로 받는다
        # emb_dim : 임베딩 벡터의 dimension을 인자로 받는다 (100)
        # dim_channel : feature map 이후로 생성되는 채널의 수를 인자로 받는다 (10)
        # kernel_wins : filter의 크기를 리스트 형태로 받는다 ([3, 4, 5])
        # nun_class : output 클래스 개수 (2)
        # '''
        
        super(TextCNN, self).__init__()
        
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        # 임베딩 설정을 위해 vocab size * embedding dimension 크기의 행렬을 만든다
        self.embed.weight.data.copy_(vocab_built.vectors)
        # copy 복사해온다는거 같다
        # fasttext 임베딩 벡터 값 가져오기
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        # input = 1, dim_channel = output, 나머지는 커널 사이즈
        # conv2d 함수에 임베딩 결과를 전달해 fillter 생성
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)
        # 오버피팅 방지
        self.fc = nn.Linear(len(kernel_wins) * dim_channel, num_class)
        # 클래스의 score 생성
        
    def forward(self, x):
        # 순전파 정의
        # output 계산 과정
        
        emb_x = self.embed(x)
        # x 의 임베딩 정보 전달
        emb_x = emb_x.unsqueeze(1)
        # 첫번째 축에 차원 추가
        # [batch_size * 570854 * 100] -> [batch_size * 1 * 570854 * 100]
        # 2차원 텍스트 데이터를 모델에 이미지처럼 넣기위해
        
        con_x = [self.relu(conv(emb_x)) for conv in self.convs]
        
        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]
        # max polling
        
        fc_x = torch.cat(pool_x, dim = 1)
        fc_x = fc_x.squeeze(-1)
        fc_x = self.dropout(fc_x)
        
        logit = self.fc(fc_x)
        
        return logit

In [6]:
# 모델 훈련

def train(model, device, train_itr, optimizer):
    
    model.train()
    corrects, train_loss = 0.0, 0
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label
        text = torch.transpose(text, 0, 1)
        # 연산을 위해 텍스트 데이터 역행렬로 변환
        target.data.sub_(1)
        # target 각 값을 1씩 줄인다
        text, target = text.to(device), target.to(device)
        # gpu
        
        optimizer.zero_grad()
        # loss gradient 할당 되는거 초기화
        logit = model(text)
        
        loss = F.cross_entropy(logit, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        # loss 누적시키기
        result = torch.max(logit, 1)[1]
        # 인덱스별로 계산된 logit 값에서 더 큰 확률 가진 클래스 저장
        corrects += (result.view(target.size()).data == target.data).sum()
        
    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)
    
    return train_loss, accuracy

In [7]:
# 모델 평가

def evaluate(model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0, 0
    
    for batch in itr:
        text, target = batch.text, batch.label
        target = batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)
        
        test_loss += loss.item()
        result = torch.max(logit, 1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()
        
    test_loss /= len(itr.dataset)
    accuracy = 100.0 * corrects / len(itr.dataset)
    
    return test_loss, accuracy

In [8]:
# 모델 학습 성능 확인

model = TextCNN(vocab, 100, 10, [3, 4, 5], 2).to(device)
print(model)

optimizer = optim.Adam(model.parameters(), lr = 0.001)

best_test_acc = -1

for epoch in range(1, 21):
    tr_loss, tr_acc = train(model, device, train_iter, optimizer)
    
    print('train epoch : {} \t Loss : {} \t Accuracy : {}%'.format(epoch, tr_loss, tr_acc))
    
    val_loss, val_acc = evaluate(model, device, validation_iter)
    
    print('valid epoch : {} \t Loss : {} \t Accuracy : {}%'.format(epoch, val_loss, val_acc))
    
    if val_acc > best_test_acc:
        best_test_acc = val_acc
        
        print('model saves at {} accuracy'.format(best_test_acc))
        torch.save(model.state_dict(), 'TextCNN_Best_Validation.pt')
        
    print('-' * 20)

TextCNN(
  (embed): Embedding(74334, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(5, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)
train epoch : 1 	 Loss : 0.09122291428824275 	 Accuracy : 51.207794189453125%
valid epoch : 1 	 Loss : 0.08657938344620518 	 Accuracy : 52.64460754394531%
model saves at 52.64460754394531 accuracy
--------------------
train epoch : 2 	 Loss : 0.08686686824614834 	 Accuracy : 52.100341796875%
valid epoch : 2 	 Loss : 0.08651517183831801 	 Accuracy : 52.595333099365234%
--------------------
train epoch : 3 	 Loss : 0.08666141512105724 	 Accuracy : 52.72187805175781%
valid epoch : 3 	 Loss : 0.08563440170611958 	 Accuracy : 54.291954040527344%
model saves at 54.291954040527344 accuracy
--------------------
train epoch : 4 	 Lo